# Cycling Demand

![imgs](images/bikes_in_street.png)

## Table of Contents

1. Overview
2. Dependencies
3. Data
4. Model Training
5. Model Evaluation
6. Summary
7. Exercises

## 1. Overview

When we travel, matter where you end up around the globe, chances are that we will end up 
stopping by a city with a bikes sharing program in place. I, personally, have had the privilege 
of renting bikes by the minute in Washington DC, New York City, London, Lyon, and a few other, 
places. But, how do the governments or companies in charge of this kind of service 
- figure out where a docking station should be located at? or,
- how many bikes should each station have? or,
- how quickly will they be rented and/or replenished?
- ...

All of these are important questions, and if the person or team in charge of figuring out the answers 
is using machine learning to figure this out, chances are, a "put 10 bikes here and 20 there, trust me 
the algorithm says it" won't suffice. Context, and sometimes persuasive arguments, can go a long way.

In this example, we will explain the behavior of a regression model on the Bike rentals[1] dataset. We will show how to calculate the partial dependence (PD) and the individual conditional expectation (ICE) to determine the feature effects on the model.

We will follow the example from the PDP chapter of the Interpretable Machine Learning[2] book and use the cleaned version of the dataset from the github repository.

## 2. Dependencies

Here are the packages we will be using in this notebook.

- `scikit-learn`
- `pandas`
- `joblib`
- `matplotlib`
- `alibi`

In [ ]:
!pip install scikit-learn pandas joblib matplotlib alibi numpy rich

In [ ]:
from alibi.explainers import PartialDependence, plot_pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from os.path import join
import pandas as pd
import numpy as np

## 3. Data

We will be using an adapted version of the [bikes sharing dataset from Washington DC](http://archive.ics.uci.edu/dataset/275/bike+sharing+dataset). This version was taken from Christoph Molnar's excellent book, Interpretable Machine Learning, and this 
notebook was written by some of my wonderful colleagues and adapted by me. 

The variables include:
- season
- yr
- mnth
- holiday
- weekday
- workingday
- weathersit
- temp
- hum
- windspeed
- cnt
- days_since_2011

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/christophM/interpretable-ml-book/master/data/bike.csv')
df.head()

We will be doing a bit of work (massaging) the features before creating a model so let's get their names first.

In [ ]:
feature_names = df.columns.tolist()
feature_names.remove('cnt')

We need to separate the categoricals from the numerical to add the transformation process to a 
well-defined pipeline. In addition, we will also need the index of each.

In [ ]:
categorical_columns_names   = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
categorical_columns_indices = [feature_names.index(cn) for cn in categorical_columns_names]
numerical_columns_indices   = [feature_names.index(fn) for fn in feature_names if fn not in categorical_columns_names]

categorical_columns_indices, numerical_columns_indices

Time to split the data. Even though you will see 0.20 below, feel free to switch the number to whichever split you'd like.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df[feature_names], df['cnt'], test_size=0.2, random_state=0
)

In [ ]:
X_train.shape

We need to change the string or object columns like season and holiday into ordinal variables 
to make them easier to work with our algorithms. Let's do that with the help of the 
`OrdinalEncoder()` class from `scikit-learn`. After we fit our training set to it we can transform 
both the training and the test set in the same fashion.

In [ ]:
oe = OrdinalEncoder().fit(X_train[categorical_columns_names])
oe

Here are the categories taken by our object.

In [ ]:
categorical_names = {i: list(v) for (i, v) in zip(categorical_columns_indices, oe.categories_)}
categorical_names

In [ ]:
X_train.loc[:, categorical_columns_names] = oe.transform(X_train[categorical_columns_names])
X_test.loc[:, categorical_columns_names]  = oe.transform(X_test[categorical_columns_names])

In [ ]:
X_train[categorical_columns_names].head()

In [ ]:
oe.categories_

Next, we can to also create dummy variables from our categorical data. 

In [ ]:
cat_transf = OneHotEncoder(
    categories=[range(len(x)) for x in categorical_names.values()],
    handle_unknown='ignore',
)
cat_transf

Lastly, our numerical values will be scaled using the `StandardScaler` class of `scikit-learn`.

In [ ]:
# define numerical standard sclaer
num_transf = StandardScaler()

We can now create a `ColumnTransformer` pipeline with the one-hot encoder and the standard scaler 
classes so that we can transform new inputs on the fly as they arrive.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transf, categorical_columns_indices),
        ('num', num_transf, numerical_columns_indices),
    ],
    sparse_threshold=0
)

In [ ]:
# fit preprocessor
preprocessor.fit(X_train.values)

In [ ]:
X_train_ohe = preprocessor.transform(X_train.values)
X_test_ohe  = preprocessor.transform(X_test.values)
X_train_ohe.shape, X_train_ohe[:2, :]

Time to train our model. 😎

## 4. Model Training

For our model, we will be using Random Forests.

> Random forests are an ensemble machine learning method used for classification and regression.

If you have never used random forests, here's an analogy that might help. Imagine you're consulting 
a crowd to predict tomorrow's weather. Asking one person might give you a bad prediction. But asking 
many people and combining their opinions gives a better result overall, and this is the idea behind random forests:

1. Build multiple decision trees on random subsets of the data. 
2. Make each tree vote on the outcome and take the most popular vote.

    It's similar to asking 100 randomly chosen people to each make a weather prediction based on limited 
    info. Then combining all their votes to get the consensus. More specifically, each decision tree is 
    trained on a random sample of features and data points. This introduces variance into the trees, so 
    they make somewhat different predictions. We could think of each person in the crowd having access to 
    slightly different weather data before making their prediction. 

3. Finally, all the trees vote and the random forest outputs the majority voted class as the prediction.

This ensemble approach reduces overfitting compared to using just one decision tree. It's unlikely 100 
people would all be wrong in the same way. The crowd wisdom improves predictions. In essence, random forests 
combine multiple decision trees to deliver accurate, robust predictions like a wise crowd. The randomness 
reduces overfitting and captures more of the complexity in the data.

In [ ]:
predictor = RandomForestRegressor(random_state=0)
predictor.fit(X_train_ohe, y_train)

In [ ]:
print('Train score: %.2f' % (predictor.score(X_train_ohe, y_train)))
print('Test score: %.2f'  % (predictor.score(X_test_ohe,  y_test)))

In [ ]:
pipe = Pipeline([('preprocess', preprocessor), ('model', predictor)])
pipe

## 5. Model Evaluation

The method we will evaluate in the section is called partial dependence plots.

> Partial dependence plots in machine learning are like snapshots of how one specific feature 
affects a model's predictions. Imagine you have a car and want to know how its speed impacts 
fuel efficiency. A partial dependence plot shows how changing the car's speed (while keeping 
everything else constant) affects its gas mileage, helping you understand the relationship 
between speed and fuel efficiency.

Imagine you have a model that predicts house prices based on size, location, etc. You 
want to know - how does size affect the predicted price?

1. Fix all the other features except size. So keep location, number of bedrooms, etc. the same. 
2. Feed the model different values for size only, keeping everything else fixed. Record 
the predicted price for each size value.

    This is like asking: "If I only change the size, how would that impact the predicted price?"

3.  Plot the size values versus the predicted prices. The curve shows the partial 
dependence of price on size.

An analogy is baking a cake. To understand how egg amount affects cake taste:

1. Fix the flour, sugar, etc. amounts 
2. Bake cakes varying only the egg amount
3. Plot taste vs. egg amount 

The curve illustrates the partial dependence of taste on eggs specifically.

So in essence, partial dependence isolates one feature to study its marginal effect 
on predictions. By ignoring interactions with other features, we can better understand 
each feature's individual relationship with the target.

Let's get started with partial dependance.

In [ ]:
explainer = PartialDependence(
    predictor=pipe.predict, feature_names=feature_names, 
    target_names=['Number of bikes'], categorical_names=categorical_names
)
explainer

Let's pick a few features of interest. Feel free to change and experiment withe the ones below.

In [ ]:
features = [
    feature_names.index('temp'),      feature_names.index('hum'), 
    feature_names.index('windspeed'), feature_names.index('season')
]

In [ ]:
# compute explanations
exp = explainer.explain(
    X=X_train.values, features=features, kind='average'
)
exp

In [ ]:
plot_pd(
    exp=exp, n_cols=2, sharey='row', fig_kw={'figheight': 10, 'figwidth': 15}
);

A limitation of PDPs is that they show the average marginal effect of a feature on the predicted 
response. But this could hide interactions that cause the effect of the feature to differ across 
individual data points. We can use the Individual Conditional Expectation sampling, or ICE, to 
fix this.

> ICE (Individual Conditional Expectation) sampling is a technique used with partial dependence 
plots (PDPs) to help account for interaction effects between features. It aims to uncover these 
interactions by creating one PDP per data point, conditional on the values of the other 
features for that individual point.

In [ ]:
exp = explainer.explain(
    X=X_train.values, features=features, kind='both'
)

The explainer above works as follows:

1. We take a single row of data
2. Then vary the feature of interest, say, `windspeed`, keeping the other features fixed at that row's values 
3. Record the prediction at each value 
4. Repeat for many rows
5. Average the conditional PDPs across rows

Now the PDP shows the isolated effect of the feature for specific combinations of the other features. Interactions 
and heterogeneity can be uncovered. For example, the effect of size on home price may differ strongly based on location. ICE sampling creates PDPs conditional on location to reveal these interactions.

Let's add a seed for the ICE sampling.

In [ ]:
np.random.seed(13)

In [ ]:
plot_pd(
    exp=exp, n_cols=2, n_ice=70, sharey='row', 
    center=True, fig_kw={'figheight': 10, 'figwidth': 15}
);

What if we wanted to observe two features at the same time to evaluate their combined effect on 
target variable while holding everything else constant? This is doable with alibi so let's pick 
some features and explain them visually.

In [ ]:
features = [
    (feature_names.index('temp'), feature_names.index('windspeed')),
    (feature_names.index('mnth'), feature_names.index('weathersit')),
    (feature_names.index('season'), feature_names.index('temp'))
]

In [ ]:
exp = explainer.explain(
    X=X_train.values, features=features, kind='average', grid_resolution=25
)

In [ ]:
plot_pd(
    exp=exp, n_cols=1, fig_kw={'figheight': 10, 'figwidth': 10}
);

## 8. Conclusion

1. Partial dependence plots are visualizations that help us understand how a single feature in a 
machine learning model influences its predictions, holding all other features constant. In a bike 
sharing scenario, it could show how temperature affects the number of bike rentals while keeping 
other factors like humidity and day of the week fixed.
2. Using Alibi Explain or plain matplotlib, we can create partial dependence plots that showcase 
the impact of individual features on model predictions, providing insights into how changes in 
specific variables affect bike rental predictions.
3. ICE (Individual Conditional Expectation) sampling is a technique used in conjunction with partial 
dependence plots to create multiple plots for different instances or samples from the dataset, 
offering a more comprehensive view of feature interactions and model behavior.
4. We need partial dependence plots to gain transparency into our machine learning models. In bike 
sharing or similar applications, where understanding how factors like weather or time affect rental 
demand, it is important to visually inspect the outputs of our models for operational decision-making 
and model trustworthiness.

## 9. Exercises

Inspecting Feature Interactions on Medical Cost Data

- Load the [medical cost prediction dataset](https://www.kaggle.com/datasets/mirichoi0218/insurance).
- Train a regression model to predict costs. 
- Use Alibi's Partial Dependance explainer, or another one of your choosing, to observe the 
interactions between different variables and the target variable. 
- Use ICE sampling to see where the average prediction lands after changing the value of your chose features.

The key is tying to build intuition for examining the models. ৻(  •̀ ᗜ •́  ৻)